# Scratch and old functions

In [ ]:
#isochronous transform for one point i
def isochronous_transform_process(t_serie, v_serie, dt, i) :
    t_i  = i*dt
    m  = 0
    t1 = -1
    while(t1 <= t_i) :
        m += 1;
        t1 = t_serie[m]
    v1 = v_serie[m]
    t0 = t_serie[m-1]
    v0 = v_serie[m-1]
    v_i  = lin_interp(v1,t1,v0,t0,t_i)
    return v_i
#parallel isochronous transform over iso time ti
def parallel_isochronous_transform(t_serie, v_serie, dt) :
    T_end    = t_serie[-1]
    Nt_iso   = int(T_end/dt)-1
    v_iso    = Parallel(n_jobs=-2)(
        delayed(isochronous_transform_process)
        (t_serie, v_serie, dt, i) for i in range(Nt_iso))
    return v_iso

In [ ]:
#Dictionnary to store isochronous transforms with various dt
v_iso  = {}; vname = DIR+'Processed_data/v_iso.dat';
t_iso  = {} #time (linspace(0,T,dt))
Nt_iso = {} #number of points
dt_iso = {}; dtname = DIR+'Processed_data/dt_iso.dat';
RERUN  = True
if  (os.path.isfile(vname) and os.path.isfile(dtname)) and (not RERUN) :
    print('Load v_iso.dat...')
    v_iso = Load_data(vname)
    print('Load dt_iso.dat...')
    dt_iso = Load_data(dtname)
    print('Done.')
RESAVE = False
for key in vDNS :
    if key not in v_iso :
        N_ = int(0.001*len(tDNS[key]))
        print("missing " + key + "data..")
        RESAVE = True
        dt = dtDNS_mean[key]
        start = time.time()
        v_iso [key]  = parallel_isochronous_transform(tDNS_ssl[key][0:N_], vDNS_ssl[key][0:N_], dt)
        Nt_iso[key]  = len(v_iso[key])
        t_iso [key]  = np.linspace(0,dt*Nt_iso[key],Nt_iso[key])
        dt_iso[key]  = [dt]
        end = time.time()
        print('\tdt = '+str(dt))
        print('\t Nt_iso = '+str(Nt_iso[key]))
        print('\tTime needed \t: '+str(end-start))
if RESAVE :
    print('Saving data...')
    Save_data(v_iso,vname)
    Save_data(dt_iso,dtname)
    print('Done.')
else : print('All data seem to be loaded.')